In [ ]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,BitsAndBytesConfig,pipeline,default_data_collator
from peft import get_peft_model,LoraConfig
import datasets
import torchmetrics
import torch
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup

cache_dir='/proj/ciptmp/ix05ogym/.cache/'


In [ ]:
#model = AutoModelForMaskedLM.from_pretrained('albert-base-v2',cache_dir=cache_dir)
toknizer = AutoTokenizer.from_pretrained('albert-base-v2',cache_dir=cache_dir)

In [ ]:
mydata = pd.read_pickle('input_text.pkl')
#mydata.reset_index(inplace=True)
mydata

In [ ]:
mydata['element']

In [ ]:
MASK = '[MASK]'
def get_t(MASK):
   t = f"i like to {MASK}." #hello i am [MASK] model.
   return t
t = get_t(MASK)
h = "this text is about traveling."
print(t+h)
inp = toknizer(t+h,return_tensors='pt')
print(inp['input_ids'])
with torch.no_grad():
   o =  model(**inp)
    



In [ ]:
index = (inp['input_ids']== toknizer.mask_token_id).int().argmax().item()
print(index ,o.logits.shape)
token_id = o.logits[0,index,:].argsort(descending=True)
r = toknizer.convert_ids_to_tokens(token_id[:5])
MASK = toknizer.convert_tokens_to_string(r[0:1])


In [ ]:
tokenizer_mnli = AutoTokenizer.from_pretrained("FacebookAI/roberta-large-mnli" , cache_dir=cache_dir)
model_mnli  = AutoModelForSequenceClassification.from_pretrained("FacebookAI/roberta-large-mnli",
                          #quantization_config=GPTQConfig(bits=4, disable_exllama=False),
                          #load_in_8bit=True,
                          cache_dir=cache_dir)

In [ ]:

t=mydata['element'][2] #f"today a cat came to me while i was playing music. i feed her and pet her. we should always take care of animals."
h = f"this html element is for {tokenizer_mnli.mask_token}."
print(t,h)

In [ ]:


inp_mnli = tokenizer_mnli(t,h,return_tensors='pt')

index = (inp_mnli['input_ids']== tokenizer_mnli.mask_token_id).int().argmax().item()
batch_input_ids = []
batch_attention_mask = []
for i in range(tokenizer_mnli.vocab_size):
    inp_mnli_copy = {key: value.clone() for key, value in inp_mnli.items()}
    inp_mnli_copy['input_ids'][0, index] = i
    batch_input_ids.append(inp_mnli_copy['input_ids'])
    batch_attention_mask.append(inp_mnli_copy['attention_mask'])
    #break

#batch = torch.vstack(batch)
batch_input_ids_tensor = torch.stack(batch_input_ids)[:,0,:]
batch_attention_mask_tensor = torch.stack(batch_attention_mask, dim=0)[:,0,:]
print(batch_input_ids_tensor.shape)
print(batch_attention_mask_tensor.shape)
model_mnli = model_mnli.to('cuda')
batch_input_ids_tensor = batch_input_ids_tensor.to('cuda')
batch_attention_mask_tensor = batch_attention_mask_tensor.to('cuda')
print(tokenizer_mnli.convert_ids_to_tokens( batch_input_ids_tensor[500]))

with torch.no_grad():
    #outputs = model_mnli.forward(input_ids=batch_input_ids_tensor, attention_mask=batch_attention_mask_tensor)
    pass
#batch[0]['input_ids'][0,index]



In [ ]:
batch_size = 512
num_batches = (batch_input_ids_tensor.shape[0]+ batch_size - 1) // batch_size 
out = []
for i in range(num_batches):
    with torch.no_grad():
        outputs = model_mnli.forward(input_ids=batch_input_ids_tensor[i*batch_size:(i+1)*batch_size]
                                     , attention_mask=batch_attention_mask_tensor[i*batch_size:(i+1)*batch_size])
        
        print(i)
        out.append(outputs)
        pass
    #print(i , batch_input_ids_tensor[i*batch_size:(i+1)*batch_size].shape)

In [ ]:
#x=torch.vstack(out)
y = []
for x in out:
   y.append( x.logits)

y = torch.vstack(y)

In [ ]:
#mm = y[:,2].argsort(descending=True)
#entailment is 2
mm = y[:,2].sort(descending=True)


In [ ]:
print(tokenizer_mnli.convert_ids_to_tokens(mm[1][:20]),mm[0][:5])

In [ ]:
#number 4 click on div
train_dataset = datasets.load_dataset('osunlp/Mind2Web',split="train",cache_dir=cache_dir)
train_dataset
d = train_dataset[4]
#var_dump(d)
print(d.keys())
print(d['actions'][0].keys())
#for x in d['actions'][0]['neg_candidates']:
#    print(x)
#print(d['actions'][0]['pos_candidates'][0].keys())
print(d['confirmed_task'])



In [84]:
d = train_dataset[500]
print(d['confirmed_task'])#['neg_candidates']
d['action_reprs']



Show car guide info for economy car and reserve.


['[button]  Cars & Services -> CLICK',
 '[link]  Car Guides -> CLICK',
 '[link]  Vehicle Info -> CLICK',
 '[link]  Reserve -> CLICK']

In [61]:
soup = BeautifulSoup(d['actions'][3]['raw_html'])
print(soup.find_all(attrs={'backend_node_id':'47749'}))

[<a backend_node_id="47749" bounding_box_rect="20,672.703125,94,21" class="ce-ajax facet facet_hover" is_clickable="true" title="Dog - Selecting this item will reload the page with filtered results.">
<span backend_node_id="40928" bounding_box_rect="20,672.703125,19,19" class="facet_checkbox" data_pw_testid_buckeye="14fbcad5-7316-455e-af71-4205fb2df872"></span>
<span backend_node_id="47752" bounding_box_rect="46,673.703125,25,15" class="facet_name">
<text backend_node_id="47753" bounding_box_rect="46,673.703125,25,15">Dog</text>
</span>
<span backend_node_id="47755" bounding_box_rect="75,673.703125,39,15" class="facetList_value">
<text backend_node_id="47756" bounding_box_rect="75,673.703125,39,15">(1,338)</text>
</span>
</a>]


In [106]:
bid = json.loads(d['actions'][1]['pos_candidates'][0]['attributes'])['backend_node_id']
print(d['actions'][1]['pos_candidates'])
soup = BeautifulSoup(lll['html'][1])#d['actions'][1]['raw_html'])
#soup = BeautifulSoup(d['actions'][3]['raw_html'])
#s = soup.find( '*' )
# Find all 'div' tags
soup = soup.find('form')
all_tags = soup.find_all()

# Remove all tags except for 'link', 'input', and 'select' tags
i=0
for element in all_tags:
    if element.name  in ['div','text']: 
    #if element.name not in ['a','link', 'input', 'select','radio','button']:
        
        element.unwrap()
    else:
        element['backend_node_id']=i
        
        if 'class' in element.attrs:
            del element['class']
            pass
        if 'bounding_box_rect' in element.attrs:
            del element['bounding_box_rect']
            pass
        if 'is_clickable' in element.attrs:
            del element['is_clickable']
            pass
        
            
        i+=1

     

s = soup.prettify()
#b = s.split("/>")
x = toknizer(s,return_tensors='pt')
print(x['input_ids'].shape[1],x['input_ids'].shape[1]/16348)


print(s)
if False:
    for i in b:
        print(i)
        print('---------------------')


[{'attributes': '{"backend_node_id": "9901", "bounding_box_rect": "770,93,270,42", "is_clickable": "true", "data_pw_testid_buckeye_candidate": "1"}', 'backend_node_id': '9901', 'is_original_target': True, 'is_top_level_target': True, 'tag': 'a'}]
1975 0.12080988500122339
<form data-testid="ApplyStep1Form" method="POST" novalidate="">
 <label backend_node_id="0" for="email">
  Email
  <span backend_node_id="1">
   *
  </span>
 </label>
 <input autocomplete="email" backend_node_id="2" data-testid="TextInput" id="email" name="email" required="" type="email" value=""/>
 <label backend_node_id="3" for="firstName">
  First Name
  <span backend_node_id="4">
   *
  </span>
 </label>
 <input backend_node_id="5" data-testid="FirstName" id="firstName" name="firstName" required="" type="text" value=""/>
 <label backend_node_id="6" for="lastName">
  Last Name
  <span backend_node_id="7">
   *
  </span>
 </label>
 <input backend_node_id="8" data-testid="LastName" id="lastName" name="lastName" requir

In [32]:
from IPython.display import display, HTML
display(HTML(s))


In [ ]:
display(HTML(s))


In [ ]:
mydata.iloc[1,:]

In [87]:
lll = pd.read_pickle('cleaned.pkl')
lll = lll.reset_index()

s = mydata[lll['url'].apply(lambda x: x.find('join.com')!=-1)]['text'][528]

In [ ]:
"""
m = set()

for d in train_dataset:
    for i in range(len(d['actions'])):
        a = d['actions'][i]
    
        #print(a['operation'])
        if len(a['pos_candidates'])>0:
            #print(a)
            m.add(a['pos_candidates'][0]['tag'])
            
            if a['pos_candidates'][0]['tag']=='div':
                #print('div')
                continue
                pass
                
    

from bs4 import BeautifulSoup

soup = BeautifulSoup(d['actions'][0]['raw_html'],'html.parser')
for e in ['a',]
soup.find_all('a')
mydata = pd.read_pickle('input_text.pkl')
mydata.pop('label')
mydata['labels'].unique()

"""

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-large-mnli" , cache_dir=cache_dir)
#x = tokenizer("[[xpath]]")
#tokenizer.convert_ids_to_tokens(x['input_ids'])
#tokenizer.additional_special_tokens

In [ ]:
#weblinx_dataset = datasets.load_dataset('',cache_dir=cache_dir)
#weblinx_dataset.keys()


In [ ]:
model_name = "FacebookAI/roberta-large-mnli"
#config = BitsAndBytesConfig(
    #load_in_4bit=True,

    #load_in_8bit=True,
    
    #bnb_4bit_quant_type="nf4",
    #bnb_4bit_use_double_quant=True,
    #bnb_4bit_compute_dtype=torch.bfloat16,
    #)


model = AutoModelForSequenceClassification.from_pretrained(model_name,
                          #quantization_config=GPTQConfig(bits=4, disable_exllama=False),
                          #load_in_8bit=True,
                          cache_dir=cache_dir).cuda()


print(model)


"""
#https://huggingface.co/docs/transformers/main/en/quantization/hqq
try:
 model_hqq = HQQModelForCausalLM.from_quantized(model_name, device_map='auto',cache_dir=cache_dir).half()#.to(torch.bfloat16) #.half()
except: 
 model_hqq = AutoHQQHFModel.from_quantized(model_name,cache_dir=cache_dir)
 
"""
 



In [ ]:
for n,p in model.named_parameters():
    p.require_grad = False
    print(n)
    


In [ ]:
tokenized_input = tokenizer("i like traveling.","this text is about x.",return_tensors='pt',truncation=True).to('cuda')
inp = torch.tensor(tokenized_input['input_ids'],requires_grad=True,dtype=torch.float32)
opt = torch.optim.SGD([inp],0.001)
for i in range(1):
    opt.zero_grad()
    o  = model(**tokenized_input)
    loss = - o.logits[0,2]
    loss.backward()
    
    opt.step()
    

In [ ]:
#pip = pipeline('zero-shot-classification',model = model,tokenizer=tokenizer,device='cuda')#pipeline_class=['email','firstname','lastname','cv'])
candidate_labels = mydata['labels'].unique()
hypothesis = [f"this text is about {label}" for label in candidate_labels]

def ppd(element):
    sequence_to_classify = element
    #print(sequence_to_classify)
    #print(candidate_labels)

    premises = [sequence_to_classify ] * len(hypothesis)
    #print(hypothesis)
    #print(premises)
    tokenized_input = tokenizer(premises,hypothesis,return_tensors='pt',truncation=True,max_length=512,padding=True).to('cuda')
    #print(x:= tokenized_input['input_ids'][0])
    #print(tokenizer.convert_ids_to_tokens(x))
    return tokenized_input
    
results = mydata['input_text'].map(ppd)

#tokenized_input =  ppd(mydata.iloc[0,:])
preds = []
with torch.no_grad():
    for d in results:
    
        o =  model(**d).logits
        positive_index = 2
        p=candidate_labels[o.argmax(0)[positive_index]]
        #print([ (candidate_labels[i],o[i,2].detach().cpu().numpy())for i in o.argsort(0,descending=True)[:,2] ])
        preds.append(p)
        #break
        pass





In [ ]:
mydata['preds']= preds

In [ ]:
mydata['labels'].hist(figsize=(20,10))

In [ ]:
print(mydata[(mydata['labels']!=preds) & ( mydata['labels']=='cv')]['element'].tolist())
#len(mydata[(mydata['labels']==preds)])/len(mydata)

In [ ]:
#x = y.to_list()
hypothesis_2 = [hypothesis ]* len(y)
print(len(x) , len(hypothesis_2))

for p,h in zip(y,hypothesis_2):
    t  = tokenizer(p,h,return_tensors='pt',truncation=False,max_length=512,padding='max_length').to('cuda')
    #input_ids.append(t['input_ids'])
    #masks.append(t['attention_mask'])
    #total.append(t)
    with torch.no_grad():
        model(**t)
    breakss


    


In [ ]:
#d = {'input_ids':torch.vstack(input_ids) , 'attention_mask':torch.vstack(masks)}
#d = default_data_collator(total)
#d
input()
import torch.utils
import torch.utils.data


#d = datasets.Dataset.from_list(total)
#d = torch.utils.data.DataLoader(total,32)
#torch.utils.data.Dataset()
d

In [ ]:
#next(iter(d))['input_ids']
#d[0]['input_ids'][0]
del d
with torch.no_grad():
    for t in total:
        model(**t)
        break

#d['input_ids'].shape


In [ ]:
d['input_ids'].shape


In [ ]:
with torch.no_grad():
    for b in d:
        print(b['input_ids'].shape)
        o = model(input_ids=b['input_ids'])
        print(o.logits)
        


In [ ]:
"""pip = pipeline('zero-shot-classification',model = model,tokenizer=tokenizer,device='cuda')#pipeline_class=['email','firstname','lastname','cv'])
sequence_to_classify = mydata['element'][1]
print(sequence_to_classify)
candidate_labels = mydata['labels'].unique()
print(candidate_labels)
o = pip(sequence_to_classify, candidate_labels)

hypothesis = [f"this text is about {label}" for label in candidate_labels]
premises = [sequence_to_classify ] * len(hypothesis)
print(hypothesis)
print(premises)
tokenized_input = tokenizer(premises,hypothesis,return_tensors='pt',truncation=True,max_length=512,padding=True).to('cuda')
print(x:= tokenized_input['input_ids'][0])
print(tokenizer.convert_ids_to_tokens(x))
with torch.no_grad():
    o =  model(**tokenized_input).logits
    pass


positive_index = 2
print(candidate_labels[o.argmax(0)[positive_index]])
[ (candidate_labels[i],o[i,2].detach().cpu().numpy())for i in o.argsort(0,descending=True)[:,2] ]
"""

In [ ]:
#model = model.eval()
#input_ids = tokenizer("create an html apply button?", return_tensors='pt').to('cuda')["input_ids"]
#outputs = model.generate(input_ids, max_new_tokens=216)
#tokenizer.decode(outputs[0])

In [ ]:
"""ArithmeticError

    for pc in a['pos_candidates']:
        #cc = json.loads(pc['attributes'])
        #t=''
        #if 'text' in cc.keys():
        #t = cc['text']
        
        #print(pc['tag'], d['action_reprs'][i])# ,'|', t)
        
        """

In GPT2 and Roberta tokenizers, the space before a word is part of a word, i.e. "Hello how are you puppetter" will be tokenized in ["Hello", "Ġhow", "Ġare", "Ġyou", "Ġpuppet", "ter"]. You can notice the spaces included in the words a Ġ here. Spaces are converted in a special character (the Ġ ) in the tokenizer prior to BPE splitting mostly to avoid digesting spaces since the standard BPE algorithm used spaces in its process (this can seem a bit hacky but was in the original GPT2 tokenizer implementation by OpenAI).

You probably have noted that the first word is a bit different because it’s lacking the first space but actually the model is trained like this and reach its best performances like this, with a special first word (see https://github.com/huggingface/transformers/issues/3788 194)
However, this behavior is a bit strange to some users because the first word is then different from the others: encoding Cats are super coolio and super coolio will not give the same tokenization (see here for instance: https://github.com/huggingface/transformers/issues/5249 122)
transformers thus provide an add_prefix_space argument to automatically add a space at the beginning if none is provided (more intuitive tokenization but slightly lower performances though).
The library used to have a complex mechanism to disable this when special tokens are used and control it dynamically. This mechanism was error-prone and this behavior is now simply activated or not at instantiation of the tokenizer (i.e. as an argument in from_pretrained ).
Also note that adding prefix space is necessary when the tokenizer is used with pre-tokenized inputs ( is_pretokenized=True ) the library has a test that raise an error if you want to encode some input with add_prefix_space=False : https://github.com/huggingface/transformers/blob/master/src/transformers/tokenization_gpt2.py#L364 147

In [38]:
$$$
from selenium import webdriver

driver = webdriver.Chrome()  # or whichever browser driver you're using
driver.get('http://your_website_url')

element = driver.find_element_by_css_selector('#your_element_selector')

# Check if the element has a click event listener
click_event = driver.execute_script("return window.getEventListeners(arguments[0]).click;", element)

if click_event:
    print("Element has a click event listener")
else:
    print("Element does not have a click event listener")

driver.quit()



"""
Things I’ve found

task prefixes matter when
1. When doing multi-task training
2. When your task similar or related to one of the supervised tasks used in T5 pre-training mixture.
Needs slightly higher LR than the default one set in Trainer, in my experiments 1e-4 and 3e-4 worked for almost all problems (classification, QA, que-gen, summ)
no need to pass decoder_input_ids to T5 yourself, just pass labels and the T5Model will prepare them for you. labels should end with eos_token. (important! This is where most of the mistakes are happening).
T5 uses pad_token as the decoder_start_token_id so when doing generation without the generate function make sure you start it with pad token.
trimming batches when training on TPU leads to very slower training.
apparently, because of sentencepiece and some possible leakage of other languages in C4 data, T5 gives somewhat sensible results for french lang. fine-tuned it on FQuAD (french version of SQuAD) for que gen and BLUE-4 against dev set was 15.
Not sure if it’s an issue or not but in some cases using label_smoothing in T5 resulted in nan loss

"""


SyntaxError: invalid syntax (2102408642.py, line 1)